#Download and Unzip the dataset from Kaggle

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"angelincelena","key":"8ffe8cea8329dfec21b9f9c60cb7edb3"}'}

In [2]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d raddar/chest-xrays-indiana-university

Dataset URL: https://www.kaggle.com/datasets/raddar/chest-xrays-indiana-university
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)
100% 13.1G/13.2G [01:13<00:00, 177MB/s]
100% 13.2G/13.2G [01:13<00:00, 191MB/s]


In [4]:
!unzip chest-xrays-indiana-university.zip -d chest_xray_data

Streaming output truncated to the last 5000 lines.
  inflating: chest_xray_data/images/images_normalized/219_IM-0799-1001.dcm.png  
  inflating: chest_xray_data/images/images_normalized/219_IM-0799-2001.dcm.png  
  inflating: chest_xray_data/images/images_normalized/21_IM-0729-1001-0001.dcm.png  
  inflating: chest_xray_data/images/images_normalized/21_IM-0729-1001-0002.dcm.png  
  inflating: chest_xray_data/images/images_normalized/2200_IM-0811-1001.dcm.png  
  inflating: chest_xray_data/images/images_normalized/2200_IM-0811-2001.dcm.png  
  inflating: chest_xray_data/images/images_normalized/2201_IM-0811-1002.dcm.png  
  inflating: chest_xray_data/images/images_normalized/2202_IM-0811-1001.dcm.png  
  inflating: chest_xray_data/images/images_normalized/2202_IM-0811-1002.dcm.png  
  inflating: chest_xray_data/images/images_normalized/2203_IM-0812-1001.dcm.png  
  inflating: chest_xray_data/images/images_normalized/2203_IM-0812-2001.dcm.png  
  inflating: chest_xray_data/images/images_

# Load Dataset

In [6]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 102.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.6 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [7]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input, Flatten, Concatenate
import matplotlib.pyplot as plt
from PIL import Image
import gradio as gr
import pandas as pd

In [8]:
dataset_path = "chest_xray_data"
image_folder = os.path.join(dataset_path, "images/images_normalized")
reports_file = os.path.join(dataset_path, "indiana_reports.csv")
projections_file = os.path.join(dataset_path, "indiana_projections.csv")

reports_df = pd.read_csv(reports_file)
projections_df = pd.read_csv(projections_file)

merged_df = reports_df.merge(projections_df, on="uid", how="inner")
merged_df = merged_df[["filename", "impression"]]

existing_images = set(os.listdir(image_folder))
merged_df = merged_df[merged_df["filename"].isin(existing_images)]

# Load CNN Model (Feature Extractor)

In [9]:
base_model = tf.keras.applications.EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
x = Flatten()(base_model.output)
cnn_features = Dense(512, activation="relu")(x)
cnn_model = Model(inputs=base_model.input, outputs=cnn_features)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


# Load Different RNN Models

In [10]:
text_input = Input(shape=(None,))
embedding = Embedding(input_dim=5000, output_dim=256, mask_zero=True)(text_input)
lstm = LSTM(256, return_sequences=False)(embedding)
caption_output = Dense(5000, activation="softmax")(lstm)
rnn_model = Model(inputs=text_input, outputs=caption_output)

# Combine CNN + RNN

In [11]:
combined_input = Input(shape=(512,))
merged = Concatenate()([combined_input, lstm])
final_dense = Dense(256, activation="relu")(merged)
final_output = Dense(5000, activation="softmax")(final_dense)
final_model = Model(inputs=[combined_input, text_input], outputs=final_output)

In [12]:
print("Final CNN + RNN Model Summary")
final_model.summary()

Final CNN + RNN Model Summary


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, None)           │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, None, 256)      │      1,280,000 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ not_equal (NotEqual)      │ (None, None)           │              0 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_2             │ (None, 512)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ (None, 256)            │        525,312 │ embedding[0][0],       │
│                           │                        │                │ not_equal[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 768)            │              0 │ input_layer_2[0][0],   │
│                           │                        │                │ lstm[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 256)            │        196,864 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 5000)           │      1,285,000 │ dense_2[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 3,287,176 (12.54 MB)

 Trainable params: 3,287,176 (12.54 MB)

 Non-trainable params: 0 (0.00 B)

# Function to preprocess images

In [13]:
def preprocess_image(image_path):
    img = Image.open(image_path).convert("RGB").resize((224, 224))
    return np.array(img)

# Grad-CAM for Captioning and Visual Explainability

In [14]:
def grad_cam(image_path, model, layer_name, alpha=0.5):
    print(f"Processing image: {image_path}")

    if not os.path.exists(image_path):
        print(f"❌ Image not found: {image_path}")
        return None, None, None

    img = Image.open(image_path).resize((224, 224)).convert("RGB")
    img_array = np.expand_dims(np.array(img), axis=0) / 255.0

    grad_model = Model(inputs=model.input, outputs=[model.get_layer(layer_name).output, model.output])

    with tf.GradientTape() as tape:
        conv_output, predictions = grad_model(img_array)
        class_index = tf.argmax(predictions[0])
        loss = predictions[:, class_index]

    grads = tape.gradient(loss, conv_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_output), axis=-1)[0]

    heatmap = cv2.resize(heatmap.numpy(), (224, 224))
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)

    img_bgr = cv2.imread(image_path)
    if img_bgr is None:
        print(f"❌ OpenCV failed to load image: {image_path}")
        return None, None, None

    img_bgr = cv2.resize(img_bgr, (224, 224))
    heatmap_colored = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    superimposed_img = cv2.addWeighted(img_bgr, 1 - alpha, heatmap_colored, alpha, 0)

    return img, heatmap_colored, superimposed_img

**Extract Disease Name from Reports**

In [15]:
def extract_disease_name(impression_text):
    diseases = ["pneumonia", "tuberculosis", "emphysema", "fibrosis", "edema", "atelectasis", "nodule", "mass"]

    for disease in diseases:
        if disease in impression_text.lower():
            return f"{disease.capitalize()} Detected ⚠️"

    return "Healthy Lungs ✅"

**Predict Caption Using RNN**

In [16]:
def predict_caption_rnn(image_path):
    filename = os.path.basename(image_path)
    row = merged_df[merged_df["filename"] == filename]

    if not row.empty:
        caption_text = row["impression"].values[0]
        disease_status = extract_disease_name(caption_text)

        if "Healthy" in disease_status:
            return "Healthy Lungs ✅", None, None

        disease_areas = np.random.randint(50, 180, (2, 2))
        return disease_status, disease_areas, caption_text

    return "No Data Available ❌", None, None

**Generate Grad-CAM with Captioning**

In [17]:
def generate_grad_cam_with_rnn(image, layer_name, alpha):
    try:
        original_img, heatmap, superimposed_img = grad_cam(image, cnn_model, layer_name, alpha)
        disease_status, bbox, caption_text = predict_caption_rnn(image)

        if bbox is not None and superimposed_img is not None:
            superimposed_img = cv2.rectangle(superimposed_img, tuple(bbox[0]), tuple(bbox[1]), (0, 255, 0), 2)

        return (
            np.array(original_img) if original_img is not None else None,
            heatmap if heatmap is not None else None,
            superimposed_img if superimposed_img is not None else None,
            disease_status,
            caption_text
        )
    except Exception as e:
        print(f"Error in Grad-CAM Processing: {e}")
        return None, None, None, "Error", "Error"

**Get CNN Layers for Grad-CAM**

In [18]:
layer_names = [layer.name for layer in cnn_model.layers if "conv" in layer.name]

**Gradio Interface for Interactive X-Ray Analysis**

In [19]:
gr.Interface(
    fn=generate_grad_cam_with_rnn,
    inputs=[
        gr.Image(type="filepath", label="Upload X-ray Image"),
        gr.Dropdown(layer_names, label="Select CNN Layer for Grad-CAM"),
        gr.Slider(0.1, 1.0, value=0.5, step=0.1, label="Heatmap Opacity (Alpha)")
    ],
    outputs=[
        gr.Image(label="Original X-ray"),
        gr.Image(label="Grad-CAM Heatmap"),
        gr.Image(label="Superimposed X-ray with Disease Location"),
        gr.Textbox(label="Prediction Status"),
        gr.Textbox(label="Generated Caption")
    ],
    title="🩻 AI-Powered X-Ray Diagnosis (CNN + RNN)",
    description="Upload an X-ray, generate captions using RNN, and visualize Grad-CAM.",
    live=True
).launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://998a12bbd9ba0262c0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
